# Lab 12-3 many to many fixed
### hihello training 
* many to many
* fixed input sequence length

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from pprint import pprint

print(tf.__version__)
tf.enable_eager_execution()

1.12.0


### Prepairing dataset
hih -> ihe, ihe -> hel, hel -> ell, ell -> llo

In [2]:
strings = 'hihello'
char_set = sorted(list(set('hihello' + 'hello')))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

In [3]:
data = []
for i in range(len(strings) -2):
    data.append(strings[i:i+3])

x_strings = data[0:4]
y_strings = data[1:5]

print(x_strings, y_strings)

['hih', 'ihe', 'hel', 'ell'] ['ihe', 'hel', 'ell', 'llo']


In [4]:
x_data = np.array(list(map(lambda strings : [char2idx.get(char) for char in strings], x_strings)))
x_data_len = np.array(list(map(lambda strings : len(strings), x_strings))).astype(np.int32)
x_data = x_data.astype(np.int32)
y_data = list(map(lambda strings : [char2idx.get(char) for char in strings], y_strings))
# y_data = np.array(list(map(lambda data : [to_categorical(token, len(char2idx))for token in data],y_data)))
y_data = np.array(y_data).astype(np.int32)
# data 형태확인
print(x_data)
print(y_data)
print(x_data_len)

[[1 2 1]
 [2 1 0]
 [1 0 3]
 [0 3 3]]
[[2 1 0]
 [1 0 3]
 [0 3 3]
 [3 3 4]]
[3 3 3 3]


In [5]:
# input
tr_dataset = tf.data.Dataset.from_tensor_slices((x_data, x_data_len, y_data))
tr_dataset = tr_dataset.shuffle(buffer_size=4)
tr_dataset = tr_dataset.batch(batch_size = 2)

print(tr_dataset)

<BatchDataset shapes: ((?, 3), (?,), (?, 3)), types: (tf.int32, tf.int32, tf.int32)>


### Creating model

In [6]:
num_classes = len(char2idx)
hidden_dim = 10
max_sequence = 3

input_dim = len(char2idx)
output_dim = len(char2idx)
one_hot = np.eye(len(char2idx))

model = keras.Sequential()
model.add(keras.layers.InputLayer(input_shape=(max_sequence,)))
model.add(keras.layers.Embedding(input_dim=input_dim, output_dim=output_dim,
                                 trainable=False, input_length=max_sequence,
                                 embeddings_initializer=keras.initializers.Constant(one_hot)))
model.add(keras.layers.SimpleRNN(units=hidden_dim, return_sequences=True))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(units=num_classes)))

### Training model

In [7]:
def loss_fn(model, x, x_len, y, max_sequence):
    masking = tf.sequence_mask(lengths=x_len, maxlen=max_sequence, dtype=tf.float32)
    return tf.contrib.seq2seq.sequence_loss(logits=model(x), targets=y, weights=masking)

lr = 0.1
epochs = 30
opt = tf.train.AdamOptimizer(learning_rate = lr)

In [8]:
for epoch in range(epochs):
    avg_tr_loss = 0
    tr_step = 0
    
    for x_mb, x_mb_len, y_mb,  in tr_dataset:
        with tf.GradientTape() as tape:
            tr_loss = loss_fn(model, x=x_mb, x_len=x_mb_len, y=y_mb, max_sequence=3)
        grads = tape.gradient(target=tr_loss, sources=model.variables)
        opt.apply_gradients(grads_and_vars=zip(grads, model.variables))
        avg_tr_loss += tr_loss
        tr_step += 1
    else:
        avg_tr_loss /= tr_step
    
    print('epoch : {:3}, tr_loss : {:.3f}'.format(epoch + 1, avg_tr_loss))

epoch :   1, tr_loss : 1.685
epoch :   2, tr_loss : 0.830
epoch :   3, tr_loss : 0.543
epoch :   4, tr_loss : 0.332
epoch :   5, tr_loss : 0.215
epoch :   6, tr_loss : 0.170
epoch :   7, tr_loss : 0.145
epoch :   8, tr_loss : 0.131
epoch :   9, tr_loss : 0.127
epoch :  10, tr_loss : 0.126
epoch :  11, tr_loss : 0.123
epoch :  12, tr_loss : 0.141
epoch :  13, tr_loss : 0.120
epoch :  14, tr_loss : 0.119
epoch :  15, tr_loss : 0.127
epoch :  16, tr_loss : 0.124
epoch :  17, tr_loss : 0.140
epoch :  18, tr_loss : 0.138
epoch :  19, tr_loss : 0.142
epoch :  20, tr_loss : 0.137
epoch :  21, tr_loss : 0.123
epoch :  22, tr_loss : 0.120
epoch :  23, tr_loss : 0.136
epoch :  24, tr_loss : 0.120
epoch :  25, tr_loss : 0.120
epoch :  26, tr_loss : 0.116
epoch :  27, tr_loss : 0.123
epoch :  28, tr_loss : 0.116
epoch :  29, tr_loss : 0.133
epoch :  30, tr_loss : 0.122


### Checking performance

In [9]:
yhat = model.predict(x_data)
yhat = np.argmax(yhat, axis=-1)
print(yhat)

[[0 1 0]
 [1 0 3]
 [0 3 3]
 [3 3 4]]


In [10]:
print('acc : {:.2%}'.format(np.mean(np.apply_along_axis(lambda row: all(row),-1, yhat == y_data))))

acc : 75.00%
